# 从零手搓中文大模型｜🚀Day03

## 数据预处理

虽然省略了数据清洗的逻辑，但是我们还是需要对数据进行预处理，以便于后续的模型训练。

包括以下两个细节：

1. 在每个文本后添加`eos`标记，以便于模型识别句子的结束。
2. 将文本转换为`数字序列`，以便于模型处理。
   
   这一步其实也可以放到模型训练的时候进行，但提前处理可以减少训练时的计算量。

### 数据集划分

解压数据集，得到`48`个jsonl文件，共计`3952863`行json数据。

我之前已经解压过了，这里把命令贴出来做个记录。

In [16]:
# !mkdir -p ../../Data/TinyStoriesChinese/raw_data/train
# !mkdir -p ../../Data/TinyStoriesChinese/raw_data/val
# !mkdir -p ../../Data/TinyStoriesChinese/processed_data/train
# !mkdir -p ../../Data/TinyStoriesChinese/processed_data/val

# !tar zxvf ../../Data/TinyStoriesChinese/TinyStories_all_data_zh.tar.gz -C ../../Data/TinyStoriesChinese/raw_data/train

我把最后一个文件`data47_zh.jsonl`（共计78538行）里切分出来4w行作为`eval`数据。

In [ ]:
# !mv ../../Data/TinyStoriesChinese/raw_data/train/data47_zh.jsonl ../../Data/TinyStoriesChinese/raw_data/eval/
# !head -n 40000 ../../Data/TinyStoriesChinese/raw_data/eval/data47_zh.jsonl > ../../Data/TinyStoriesChinese/raw_data/eval/eval.jsonl
# !tail -n +40000 ../../Data/TinyStoriesChinese/raw_data/eval/data47_zh.jsonl > ../../Data/TinyStoriesChinese/raw_data/train/data47_zh.jsonl
# !rm ../../Data/TinyStoriesChinese/raw_data/eval/data47_zh.jsonl

### 先看一条数据

In [21]:
import json

with open("../../Data/TinyStoriesChinese/raw_data/train/data00_zh.jsonl", "r") as f:
    for line in f.readlines():
        js = json.loads(line)
        print(js["story_zh"])
        break

莉莉和本是朋友。他们喜欢在公园里玩。有一天，他们在一棵大树下看到了一个秋千。莉莉想试试那个秋千。她跑到树下，爬上了秋千。
"推我，本！"她说。本轻轻地推了她一下。莉莉感到很开心。她越荡越高，笑着喊叫。
本看着莉莉。他觉得她很可爱。他也想荡秋千。他在莉莉停下来之后等着。但是莉莉没有停下来。她越荡越快。她玩得太高兴了。
"我也可以荡秋千吗，莉莉？"本问。莉莉没听到他的话。她忙着荡秋千。本觉得很难过。他走开了。
莉莉荡得太高，失去了平衡。她从秋千上摔下来，落在地上。她扭伤了脚。她哭了起来。
"哎呀，哎呀，哎呀！"她说。她在找本。她希望他能帮助她。但本不在那里。他走了。
莉莉感到很抱歉。她希望她能和本分享秋千。她希望他在那里拥抱她。她一瘸一拐地走到树下。她看到有什么东西挂在树枝上。那是本的帽子。他留给她的。
莉莉笑了。她觉得本很好。她戴上了他的帽子。她希望他会回来。她想道歉。她想再次成为朋友。


### 适配框架API

由于选择了使用[⚡️litgpt](https://github.com/Lightning-AI/litgpt/tree/main)框架进行训练，所以需要引入框架相关的`Class`和`API`来封装我们的数据准备逻辑。

这里我们可以参考[源码里集成的Tinyllama的数据预处理代码](https://github.com/Lightning-AI/litgpt/blob/main/litgpt/data/prepare_slimpajama.py)里的代码，稍作修改。

主要是需要将[Day02](../Day02/Day02.ipynb)里的`line`处理逻辑封装到`ligtgpt`的`API`中。

但在此之前我们先熟悉一下`litgpt`的Tokenizer的使用方法：

In [25]:
import torch
from litgpt import Tokenizer

litgpt_tokenizer = Tokenizer("../../References/chatglm3-6b")

这里也实验了一下结果，对比发现和上面咱们之前用原生Tokenizer处理的**结果一致**。

（不过需要注意litgpt的`Tokenizer.encode`返回的是一个torch的`Tensor`）

In [23]:
litgpt_encoded = litgpt_tokenizer.encode(
    json.loads(line)["story_zh"], eos=True
)  # 记得设置eos=True
print(litgpt_encoded)
print(litgpt_tokenizer.decode(litgpt_encoded))

tensor([30910, 56623, 56623, 54542, 50154, 31761, 31155, 31633, 31815, 54534,
        32693, 54662, 55409, 31155, 35632, 31123, 31633, 34383, 57427, 47658,
        54578, 34518, 31623, 55567, 55226, 31155, 56623, 56623, 54695, 39887,
        32437, 55567, 55226, 31155, 54790, 41309, 52624, 31123, 56856, 32660,
        55567, 55226, 31155,    13, 30955, 54834, 54546, 31123, 54613, 31404,
        30955, 36213, 31155, 54613, 36660, 54563, 54834, 43881, 32024, 31155,
        56623, 56623, 32707, 54657, 33436, 31155, 54790, 54937, 56567, 40714,
        31123, 38502, 56653, 55483, 31155,    13, 54613, 32984, 56623, 56623,
        31155, 54572, 31897, 54790, 54657, 35245, 31155, 36551, 54695, 56567,
        55567, 55226, 31155, 33152, 56623, 56623, 51556, 31797, 39055, 31155,
        31694, 56623, 56623, 31631, 51556, 31155, 54790, 54937, 56567, 54937,
        54929, 31155, 54790, 55409, 40915, 34492, 54537, 31155,    13, 30955,
        54546, 32591, 56567, 55567, 55226, 55398, 31123, 56623, 

数据处理参考上面给出的[链接](https://github.com/Lightning-AI/litgpt/blob/main/litgpt/data/prepare_slimpajama.py)，我们需要实现`prepare_slimpajama.py`里的相关函数。

In [32]:
# Copyright Lightning AI. Licensed under the Apache License 2.0, see LICENSE file.

import json
import os
import time
import numpy as np
from pathlib import Path

from litgpt.tokenizer import Tokenizer
from litgpt.data.prepare_starcoder import DataChunkRecipe
from litgpt.utils import extend_checkpoint_dir


class TinyStoriesZhDataRecipe(DataChunkRecipe):
    is_generator = True

    def __init__(self, tokenizer: Tokenizer, chunk_size: int):
        super().__init__(chunk_size)
        self.tokenizer = tokenizer

    def prepare_structure(self, input_dir):
        files = Path(input_dir).rglob("*.jsonl")
        return [str(file) for file in files]

    def prepare_item(self, filepath):

        with open(filepath, "rb") as f:
            for line in f.readlines():
                js = json.loads(line)
                story = js["story_zh"]
                # 注意这里要添加eos
                # 另外还记得吗：我们的vocab size在int16范围内，所以可以转换为uint16来节省内存
                story_ids = np.array(
                    self.tokenizer.encode(story, eos=True), dtype=np.uint16
                )
                yield story_ids


def prepare(
    input_dir: Path = Path("../../Data/TinyStoriesChinese/raw_data/train"),
    output_dir: Path = Path("../../Data/TinyStoriesChinese/processed_data/train"),
    tokenizer_path: Path = Path("../../References/chatglm3-6b"),
    chunk_size: int = (2049 * 16384),
    fast_dev_run: bool = False,
) -> None:
    from litdata.processing.data_processor import DataProcessor

    tokenizer_path = extend_checkpoint_dir(tokenizer_path)
    tokenizer = Tokenizer(tokenizer_path)
    data_recipe = TinyStoriesZhDataRecipe(tokenizer=tokenizer, chunk_size=chunk_size)
    data_processor = DataProcessor(
        input_dir=str(input_dir),
        output_dir=str(output_dir),
        fast_dev_run=fast_dev_run,
        num_workers=os.cpu_count(),
        num_downloaders=1,
    )

    start_time = time.time()
    data_processor.run(data_recipe)
    elapsed_time = time.time() - start_time
    print(f"Time taken: {elapsed_time:.2f} seconds")

#### 先用eval数据集测试

（也可以设置`fast_dev_run=True`来处理更少的数据，尤其是debug时）

执行完可以在`processed_data/eval`目录下看到生成的chunk文件。

比较一下可以发现从原先的`83m`的`.jsonl`文件压缩到了`13m`的`.bin`,

In [ ]:
prepare(
    input_dir=Path("../../Data/TinyStoriesChinese/raw_data/eval"),
    output_dir=Path("../../Data/TinyStoriesChinese/processed_data/eval"),
    tokenizer_path=Path("../../References/chatglm3-6b"),
)

#### 处理train数据集
在32核的CPU上处理`train`数据集耗时不到`1min`。

In [ ]:
prepare(
    input_dir=Path("../../Data/TinyStoriesChinese/raw_data/train"),
    output_dir=Path("../../Data/TinyStoriesChinese/processed_data/train"),
    tokenizer_path=Path("../../References/chatglm3-6b"),
)

## 小结

1. 数据预处理的逻辑主要是将文本转换为数字序列，以便于模型处理。
2. 通过litgpt的Tokenizer可以方便的实现文本到数字序列的转换。
3. litgpt提供了数据处理的API，可以方便的封装我们的数据处理逻辑。
4. 数据处理的结果可以通过压缩文件的方式减少存储空间。